In [1]:
import ott
from ott.tools.sinkhorn_divergence import sinkhorn_divergence
from ott.core.sinkhorn import sinkhorn, Sinkhorn
from ott.geometry.pointcloud import PointCloud
from ott.geometry.geometry import Geometry
import jax
import jax.numpy as jnp
from ott.core import LinearProblem
from ott.core import unbalanced_functions
import ot
import sklearn
import numpy as np
import scipy

In [6]:
jax.default_backend()

'gpu'

In [2]:
n = 1000
dim = 30

rng = jax.random.PRNGKey(0)
rng, *rngs = jax.random.split(rng, 5)
x = jax.random.uniform(rngs[0], (n, dim))
y = jax.random.uniform(rngs[1], (n, dim)) + 0.1
a = jnp.ones(n)
b = jnp.ones(n)
a_scaled = a / jnp.sum(a)
b_scaled = b / jnp.sum(b)

2022-09-05 18:19:02.991043: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-09-05 18:19:03.449125: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-09-05 18:19:03.827346: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to 

In [3]:
lp = LinearProblem(PointCloud(x, y, epsilon=1), a_scaled, b_scaled, tau_a=0.5, tau_b=0.5)

In [4]:
s = Sinkhorn()(lp)

2022-09-05 18:19:07.044763: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-09-05 18:19:07.211270: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-09-05 18:19:21.605381: W external/org_tensorflow/tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to 

In [5]:
s.reg_ot_cost

DeviceArray(-17.182905, dtype=float32)

In [217]:
tmap_ott = s.matrix

In [218]:
pc = PointCloud(x, y, epsilon=1)

In [219]:
M = pc.cost_matrix
reg = 1
reg_m = 0.5

In [220]:
tmap_stabilized = ot.sinkhorn_unbalanced(a_scaled, b_scaled, M, reg, reg_m, method='sinkhorn_stabilized',
                         numItermax=1000, stopThr=1e-6, verbose=False,
                         log=False)

In [221]:
cost_stabilized = ot.sinkhorn_unbalanced2(a_scaled, b_scaled, M, reg, reg_m, method='sinkhorn_stabilized',
                         numItermax=1000, stopThr=1e-6, verbose=False,
                         log=False)

In [222]:
tmap_unstabilized = ot.sinkhorn_unbalanced(a_scaled, b_scaled, M, reg, reg_m, method='sinkhorn',
                         numItermax=1000, stopThr=1e-6, verbose=False,
                         log=False)

In [223]:
cost_unstabilized = ot.sinkhorn_unbalanced2(a_scaled, b_scaled, M, reg, reg_m, method='sinkhorn',
                         numItermax=1000, stopThr=1e-6, verbose=False,
                         log=False)

In [224]:
cost_stabilized, cost_unstabilized, s.reg_ot_cost

(array([392.40014602]),
 array([392.4002301]),
 DeviceArray(-17.182905, dtype=float32))

In [225]:
tmap_unstabilized.sum(), tmap_stabilized.sum(), tmap_ott.sum()

(91.84217097182267, 91.84215129307975, DeviceArray(20.142618, dtype=float32))

In [226]:
np.corrcoef(tmap_unstabilized.flatten(), tmap_stabilized.flatten())

array([[1., 1.],
       [1., 1.]])

In [227]:
np.corrcoef(tmap_unstabilized.flatten(), tmap_ott.flatten())

array([[1.        , 0.99190975],
       [0.99190975, 1.        ]])

## Same procedure with non-scaled marginals

In [228]:
lp2 = LinearProblem(PointCloud(x, y, epsilon=1), a, b, tau_a=0.5, tau_b=0.5)

In [229]:
s2 = Sinkhorn()(lp2)

In [230]:
tmap_ott2 = s2.matrix

In [231]:
tmap_stabilized2 = ot.sinkhorn_unbalanced(a, b, M, reg, reg_m, method='sinkhorn_stabilized',
                         numItermax=1000, stopThr=1e-6, verbose=False,
                         log=False)

In [232]:
cost_stabilized2 = ot.sinkhorn_unbalanced2(a, b, M, reg, reg_m, method='sinkhorn_stabilized',
                         numItermax=1000, stopThr=1e-6, verbose=False,
                         log=False)

In [233]:
tmap_unstabilized2 = ot.sinkhorn_unbalanced(a, b, M, reg, reg_m, method='sinkhorn',
                         numItermax=1000, stopThr=1e-6, verbose=False,
                         log=False)

In [234]:
cost_unstabilized2 = ot.sinkhorn_unbalanced2(a, b, M, reg, reg_m, method='sinkhorn',
                         numItermax=1000, stopThr=1e-6, verbose=False,
                         log=False)

In [235]:
cost_stabilized2, cost_unstabilized2, s2.reg_ot_cost

(array([12408.78186265]),
 array([12408.78226408]),
 DeviceArray(995957.2, dtype=float32))

Here we can see that POT only reports the non-regularised OT cost

In [241]:
np.multiply(M, tmap_stabilized2).sum()

12408.781862645832

... as oppposed to OTT-JAX

In [253]:
np.multiply(M, tmap_ott2).sum()


8713.144

In [207]:
np.sum(np.dot(tmap_stabilized2, M))

10054524.811768912

In [104]:
tmap_unstabilized2.sum(), tmap_stabilized2.sum(), tmap_ott2.sum()

(1889.6570213879825, 1889.6569623935577, DeviceArray(1173.2864, dtype=float32))

In [105]:
np.corrcoef(tmap_unstabilized2.flatten(), tmap_stabilized2.flatten())

array([[1., 1.],
       [1., 1.]])

In [106]:
np.corrcoef(tmap_unstabilized2.flatten(), tmap_ott2.flatten())

array([[1.        , 0.98059842],
       [0.98059842, 1.        ]])

Check correlation of transport matrices obtained with different scale of marginals

In [254]:
np.corrcoef(tmap_unstabilized2.flatten(), tmap_unstabilized.flatten())

array([[1., 1.],
       [1., 1.]])

In [255]:
np.corrcoef(tmap_ott.flatten(), tmap_ott2.flatten())

array([[1., 1.],
       [1., 1.]])

... as expected linear scaling of the marginals does not affect the transport plan

As a comparison check the scale of the cost of balanced Sinkhorn

In [168]:
lp3 = LinearProblem(PointCloud(x, y, epsilon=1), a, b)

In [169]:
s3 = Sinkhorn()(lp3)

In [170]:
s3.reg_ot_cost

DeviceArray(245641.27, dtype=float32)

In [171]:
lp4 = LinearProblem(PointCloud(x, y, epsilon=1e-3), a, b)

In [172]:
s4 = Sinkhorn()(lp4)

In [173]:
s4.reg_ot_cost

DeviceArray(1580.634, dtype=float32)

# TODOs

- Stopping criterion of unbalanced OT should account for scale of marginals, otherwise we never converge
- maybe use stabilised implementation?
- check computation of final cost, compare with equation in Sejourne, 2019

In [201]:
ot_prob = lp2

In [202]:
f = s2.f
g = s2.g

In [203]:
lse_mode=True

In [204]:
supp_a = ot_prob.a > 0
supp_b = ot_prob.b > 0
fa = ot_prob.geom.potential_from_scaling(ot_prob.a)
if ot_prob.tau_a == 1.0:
    div_a = jnp.sum(jnp.where(supp_a, ot_prob.a * (f - fa), 0.0))
else:
    rho_a = ot_prob.epsilon * (ot_prob.tau_a / (1 - ot_prob.tau_a))
    div_a = -jnp.sum(
        jnp.where(
            supp_a, ot_prob.a * unbalanced_functions.phi_star(-(f - fa), rho_a),
            0.0
        )
    )

gb = ot_prob.geom.potential_from_scaling(ot_prob.b)
if ot_prob.tau_b == 1.0:
    div_b = jnp.sum(jnp.where(supp_b, ot_prob.b * (g - gb), 0.0))
else:
    rho_b = ot_prob.epsilon * (ot_prob.tau_b / (1 - ot_prob.tau_b))
    div_b = -jnp.sum(
    jnp.where(
        supp_b, ot_prob.b * unbalanced_functions.phi_star(-(g - gb), rho_b),
        0.0
    )
    )

# Using https://arxiv.org/pdf/1910.12958.pdf (24)
if lse_mode:
    total_sum = jnp.sum(ot_prob.geom.marginal_from_potentials(f, g))
else:
    u = ot_prob.geom.scaling_from_potential(f)
    v = ot_prob.geom.scaling_from_potential(g)
    total_sum = jnp.sum(ot_prob.geom.marginal_from_scalings(u, v))
res = div_a + div_b + ot_prob.epsilon * (
  jnp.sum(ot_prob.a) * jnp.sum(ot_prob.b) - total_sum
)


In [206]:
res, div_a, div_b, jnp.sum(ot_prob.a), jnp.sum(ot_prob.b), total_sum

(DeviceArray(245133.56, dtype=float32),
 DeviceArray(-200.08899, dtype=float32),
 DeviceArray(-201.83295, dtype=float32),
 DeviceArray(498.85, dtype=float32),
 DeviceArray(494.555, dtype=float32),
 DeviceArray(1173.2864, dtype=float32))